# Neural Network

We'll now use a Neural Network to predict the players identity.

In [1]:
%matplotlib notebook

import pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.decomposition import PCA

Using TensorFlow backend.


## We'll start with the features and the topology proposed by the last year and train the NN with it. After we'll try with our features (generated by wave instead of each balloon).

In [20]:
data = np.genfromtxt('./features/kate_data_julien_sarah.csv', delimiter=',')
np.random.shuffle(data)

In [21]:
training_ratio = 0.85
l = len(data)
X = data[:,:-1]
y = data[:,-1]
X_train = X[:int(l*training_ratio)]
X_test = X[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

In [22]:
y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

# Dimensionality reduction with PCA

In [23]:
mu = X_train.mean(axis=0)
U,s,V = np.linalg.svd(X_train - mu, full_matrices=False)
Zpca = np.dot(X_train - mu, V.transpose())

Rpca = np.dot(Zpca[:,:2], V[:2,:]) + mu    # reconstruction
err = np.sum((X_train-Rpca)**2)/Rpca.shape[0]/Rpca.shape[1]
print('PCA reconstruction error with 2 PCs: ' + str(round(err,3)));
print(max(Zpca[:,0]))
print(min(Zpca[:,0]))
print(max(Zpca[:,1]))
print(min(Zpca[:,1]))

print(np.argmax(Zpca[:,0]))
print(np.argmax(Zpca[:,1]))

PCA reconstruction error with 2 PCs: 7.471
2234.6621633472378
-22.639794956373947
128.40015756257122
-369.87814850662505
94
94


# Building and training of a dnn

In [24]:
m = Sequential()
m.add(Dense(150,  activation='relu', input_shape=(105,)))
#m.add(Dense(150,  activation='relu')) 
m.add(Dense(150,  activation='relu'))
m.add(Dense(150,  activation='relu'))
m.add(Dense(50,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/20
1360/1360 [==============================] - 1s 605us/step - loss: 0.6897 - acc: 0.5272 - val_loss: 0.6454 - val_acc: 0.4250
Epoch 2/20
1360/1360 [==============================] - 0s 168us/step - loss: 0.5857 - acc: 0.6199 - val_loss: 0.5939 - val_acc: 0.5500
Epoch 3/20
1360/1360 [==============================] - 0s 173us/step - loss: 0.5346 - acc: 0.7515 - val_loss: 0.5569 - val_acc: 0.7458
Epoch 4/20
1360/1360 [==============================] - 0s 165us/step - loss: 0.5692 - acc: 0.7603 - val_loss: 0.5654 - val_acc: 0.5750
Epoch 5/20
1360/1360 [==============================] - 0s 216us/step - loss: 0.5196 - acc: 0.7831 - val_loss: 0.5552 - val_acc: 0.6583
Epoch 6/20
1360/1360 [==============================] - 0s 169us/step - loss: 0.5210 - acc: 0.7596 - val_loss: 0.5590 - val_acc: 0.5958
Epoch 7/20
1360/1360 [==============================] - 0s 171us/step - loss: 0.4939 - acc: 0.7816 - val_loss: 0.6841 - val_acc: 0.7333
E

In [25]:
y_pred = m.predict(X_test)

In [26]:
accuracy = m.evaluate(X_test, y_test)[1]
print("Précision old features: %.2f" % accuracy)

240/240 [==============================] - 0s 37us/step
Précision old features: 0.81


## Now let's try with our features and with a different topology. Since we have computed a 12 dimension feature-vector, it would make no sense to use layers with more than 100 neurons as the last year group did.

In [17]:
X = np.genfromtxt('./features/features_wave_julian_sarah.csv', delimiter=',')
y = np.genfromtxt('./features/output_wave_julian_sarah.csv', delimiter=',')

p = np.random.permutation(len(X))
X, y = X[p], y[p]

training_ratio = 0.85
l = len(y)
X_train = X[:int(l*training_ratio)]
X_test = X[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

In our case we have 12 dimensions to features instead. Let's apply the NN directly without a PCA to compare later

In [18]:
m = Sequential()
m.add(Dense(15,  activation='relu', input_shape=(12,)))
m.add(Dense(15,  activation='relu'))
m.add(Dense(15,  activation='relu'))
m.add(Dense(4,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/20
1360/1360 [==============================] - 1s 466us/step - loss: 0.5555 - acc: 0.7846 - val_loss: 0.5473 - val_acc: 0.8167
Epoch 2/20
1360/1360 [==============================] - 0s 86us/step - loss: 0.5331 - acc: 0.7926 - val_loss: 0.5314 - val_acc: 0.7958
Epoch 3/20
1360/1360 [==============================] - 0s 96us/step - loss: 0.5195 - acc: 0.7993 - val_loss: 0.5249 - val_acc: 0.8000
Epoch 4/20
1360/1360 [==============================] - 0s 85us/step - loss: 0.5105 - acc: 0.8037 - val_loss: 0.5086 - val_acc: 0.8167
Epoch 5/20
1360/1360 [==============================] - 0s 84us/step - loss: 0.4987 - acc: 0.8000 - val_loss: 0.4935 - val_acc: 0.8208
Epoch 6/20
1360/1360 [==============================] - 0s 81us/step - loss: 0.4950 - acc: 0.8000 - val_loss: 0.4858 - val_acc: 0.8208
Epoch 7/20
1360/1360 [==============================] - 0s 87us/step - loss: 0.4772 - acc: 0.8074 - val_loss: 0.4777 - val_acc: 0.8333
Epoch 8

In [19]:
accuracy = m.evaluate(X_test, y_test)[1]
print("Précision New features: %.2f" % accuracy)

240/240 [==============================] - 0s 29us/step
Précision New features: 0.88


# We got a precision of 88%, which is better than the 81% of the last year. But we would need to do an average to compare

Let's apply PCA now to reduce the dimension to 3

In [27]:
model_pca3 = PCA(n_components=3)

# On entraîne notre modèle (fit) sur les données
model_pca3.fit(X)

# On applique le résultat sur nos données :
X_reduced3 = model_pca3.transform(X)

In [28]:
training_ratio = 0.85
l = len(y)
X_train = X_reduced3[:int(l*training_ratio)]
X_test = X_reduced3[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

In [29]:
m = Sequential()
m.add(Dense(20,  activation='relu', input_shape=(3,)))
#m.add(Dense(20,  activation='relu')) 
m.add(Dense(20,  activation='relu'))
m.add(Dense(20,  activation='relu'))
m.add(Dense(5,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/20
1360/1360 [==============================] - 1s 555us/step - loss: 0.5776 - acc: 0.7301 - val_loss: 0.5513 - val_acc: 0.7500
Epoch 2/20
1360/1360 [==============================] - 0s 91us/step - loss: 0.4966 - acc: 0.7831 - val_loss: 0.5360 - val_acc: 0.7583
Epoch 3/20
1360/1360 [==============================] - 0s 87us/step - loss: 0.4722 - acc: 0.7860 - val_loss: 0.5485 - val_acc: 0.7333
Epoch 4/20
1360/1360 [==============================] - 0s 88us/step - loss: 0.4613 - acc: 0.7897 - val_loss: 0.5361 - val_acc: 0.7583
Epoch 5/20
1360/1360 [==============================] - 0s 86us/step - loss: 0.4488 - acc: 0.7949 - val_loss: 0.5459 - val_acc: 0.7458
Epoch 6/20
1360/1360 [==============================] - 0s 87us/step - loss: 0.4395 - acc: 0.7963 - val_loss: 0.5291 - val_acc: 0.7375
Epoch 7/20
1360/1360 [==============================] - 0s 102us/step - loss: 0.4388 - acc: 0.7934 - val_loss: 0.5254 - val_acc: 0.7417
Epoch 

Using a PCA doesn't seems to improve the accuracy, but the accuracy seems to be more stable with respect to the epochs

## We conclude that the Neural Network gives a better precision that our previous alorithms. Computing the features by wave instead of by balloons also seems to improve the Neural Network precision. The only disavantage is that we get less data by doing so. If few data is available, it might be better to use the features computed by balloon instead